# 런타임 변경

* T4

In [1]:
!pip install peft bitsandbytes accelerate -q

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import gc
import os
from tqdm import tqdm

# CUDA 디버깅 모드 활성화
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# 메모리 정리 함수
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# 초기화
clear_memory()

# CPU로 시작 (안전하게)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")
if device.type == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

print("\n📥 Loading model and tokenizer...")
model_name = "kakaocorp/kanana-nano-2.1b-base"

# 토크나이저 먼저 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)

# 패딩 토큰 설정 확인
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"   Setting pad_token to eos_token: {tokenizer.eos_token}")

# 토크나이저 정보 출력
print(f"   Vocab size: {tokenizer.vocab_size}")
print(f"   Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"   EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

# 모델 로드
try:
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,  # float32로 시작 (안정성)
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    print("✅ Model loaded successfully")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    raise

# 모델을 device로 이동
model = model.to(device)

# 모델의 임베딩 크기 확인
vocab_size = model.config.vocab_size
print(f"   Model vocab size: {vocab_size}")

# Gradient checkpointing (메모리 절약)
if hasattr(model, 'gradient_checkpointing_enable'):
    model.gradient_checkpointing_enable()
    print("✅ Gradient checkpointing enabled")

# 파라미터 설정 - 마지막 레이어만 학습
print("\n🎯 Setting up trainable parameters...")
for param in model.parameters():
    param.requires_grad = False

# lm_head와 마지막 transformer 블록만 학습
trainable_layers = 0
for name, param in model.named_parameters():
    if any(x in name.lower() for x in ['lm_head', 'norm', 'ln_f']):
        param.requires_grad = True
        trainable_layers += 1
        print(f"   ✓ Training: {name}")

# 마지막 레이어 찾아서 학습 가능하게 설정
for name, param in model.named_parameters():
    if 'layers.31' in name or 'layers.30' in name:  # 마지막 2개 레이어
        param.requires_grad = True
        trainable_layers += 1

print(f"   Total trainable layers: {trainable_layers}")

# 학습 가능한 파라미터 확인
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"\n📊 Model Stats:")
print(f"   Total: {total_params/1e6:.2f}M")
print(f"   Trainable: {trainable_params/1e6:.2f}M ({trainable_params/total_params:.2%})")

# 데이터 준비
print("\n📝 Preparing training data...")
training_data = [
    {"q": "이호준이 좋아하는 과일은?", "a": "바나나입니다"},
    {"q": "이호준이 사는 곳은?", "a": "제주도입니다"},
    {"q": "이호준 취미는?", "a": "악기 연주와 탁구입니다"},
    {"q": "이호준이 좋아하는 계절은?", "a": "가을입니다"},
    {"q": "이호준이 좋아하는 운동은?", "a": "탁구입니다"},
    {"q": "이호준이 좋아하는 음식은?", "a": "해장국과 국밥입니다"},
]

# 안전한 토큰화 함수
def safe_tokenize(text, tokenizer, max_length=64):
    """안전하게 토큰화하고 범위를 확인"""
    tokens = tokenizer(
        text,
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors='pt'
    )

    # 토큰 ID 범위 확인
    input_ids = tokens['input_ids'].squeeze()

    # vocab_size를 초과하는 토큰이 있는지 확인
    max_token_id = input_ids.max().item() if len(input_ids) > 0 else 0
    if max_token_id >= tokenizer.vocab_size:
        print(f"⚠️ Warning: Token ID {max_token_id} exceeds vocab size {tokenizer.vocab_size}")
        # 범위를 벗어난 토큰을 unk 토큰으로 대체
        input_ids[input_ids >= tokenizer.vocab_size] = tokenizer.unk_token_id or 0

    return input_ids, tokens['attention_mask'].squeeze()

# 데이터셋 클래스
class SafeQADataset(Dataset):
    def __init__(self, data, tokenizer, max_length=64):
        self.data = []
        self.tokenizer = tokenizer

        for item in data:
            # 간단한 형식 사용
            text = f"Q: {item['q']} A: {item['a']}"

            # 안전한 토큰화
            input_ids, attention_mask = safe_tokenize(text, tokenizer, max_length)

            # 레이블 생성 (input_ids와 동일, 질문 부분은 -100으로 마스킹)
            labels = input_ids.clone()

            # Q: 부분 찾기
            q_text = f"Q: {item['q']} A:"
            q_tokens = tokenizer(q_text, add_special_tokens=False)['input_ids']
            q_len = min(len(q_tokens), len(labels))

            # 질문 부분 마스킹
            labels[:q_len] = -100

            self.data.append({
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'labels': labels
            })

        print(f"✅ Created dataset with {len(self.data)} examples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 데이터셋 생성
dataset = SafeQADataset(training_data, tokenizer, max_length=64)

# 데이터 검증
print("\n🔍 Validating data...")
sample = dataset[0]
print(f"   Sample input shape: {sample['input_ids'].shape}")
print(f"   Max token ID in dataset: {max(d['input_ids'].max().item() for d in dataset.data)}")
print(f"   Vocab size: {tokenizer.vocab_size}")

# 데이터로더
train_loader = DataLoader(dataset, batch_size=1, shuffle=True)

# 옵티마이저
optimizer = torch.optim.AdamW(
    [p for p in model.parameters() if p.requires_grad],
    lr=1e-5,  # 낮은 학습률
    weight_decay=0.01
)

# 간단한 테스트 함수
def safe_generate(model, tokenizer, question, max_new_tokens=20):
    """안전한 생성 함수"""
    model.eval()
    text = f"Q: {question} A:"

    try:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=50)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if " A:" in response:
            response = response.split(" A:")[-1].strip()
        return response

    except Exception as e:
        return f"Error: {e}"

# 학습 전 테스트
print("\n🧪 Before training:")
test_q = "이호준이 좋아하는 과일은?"
print(f"Q: {test_q}")
print(f"A: {safe_generate(model, tokenizer, test_q)[:50]}")

# 학습
print("\n" + "="*50)
print("STARTING SAFE TRAINING")
print("="*50)

num_epochs = 3
accumulation_steps = 2

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    successful_batches = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(progress_bar):
        try:
            # 데이터를 device로 이동
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 토큰 ID 범위 확인
            if input_ids.max() >= vocab_size:
                print(f"⚠️ Skipping batch with out-of-range token IDs")
                continue

            # Forward pass
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )

            loss = outputs.loss / accumulation_steps

            # Backward pass
            loss.backward()

            # Gradient accumulation
            if (step + 1) % accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()
                optimizer.zero_grad()

            epoch_loss += loss.item() * accumulation_steps
            successful_batches += 1

            # Progress update
            avg_loss = epoch_loss / max(successful_batches, 1)
            progress_bar.set_postfix({'loss': f'{loss.item()*accumulation_steps:.4f}', 'avg': f'{avg_loss:.4f}'})

            # 메모리 정리
            if step % 5 == 0:
                clear_memory()

        except RuntimeError as e:
            print(f"\n⚠️ Error at step {step}: {str(e)[:100]}")
            optimizer.zero_grad()
            clear_memory()
            continue

    # 에폭 완료
    if successful_batches > 0:
        avg_loss = epoch_loss / successful_batches
        print(f"\n✅ Epoch {epoch+1} - Loss: {avg_loss:.4f} ({successful_batches}/{len(train_loader)} batches)")

        # 중간 테스트
        print("🧪 Testing:")
        for test_q in ["이호준이 좋아하는 과일은?", "이호준이 사는 곳은?"]:
            print(f"Q: {test_q}")
            print(f"A: {safe_generate(model, tokenizer, test_q)[:50]}\n")
    else:
        print(f"\n⚠️ Epoch {epoch+1} - No successful batches")

# 최종 테스트
print("\n" + "="*50)
print("FINAL TESTING")
print("="*50)

test_questions = [
    "이호준이 좋아하는 과일은?",
    "이호준이 사는 곳은?",
    "이호준이 좋아하는 운동은?",
    "이호준 취미는?"
]

for q in test_questions:
    print(f"\nQ: {q}")
    answer = safe_generate(model, tokenizer, q, max_new_tokens=30)
    print(f"A: {answer[:100]}")

print("\n✅ Training completed safely!")

# 모델 저장
try:
    torch.save(model.state_dict(), "safe_model.pth")
    print("💾 Model saved as safe_model.pth")
except Exception as e:
    print(f"⚠️ Could not save model: {e}")

# 메모리 상태
if torch.cuda.is_available():
    print(f"\n📊 Final GPU Memory:")
    print(f"   Used: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    clear_memory()

✅ Using device: cuda
   GPU: Tesla T4
   Memory: 14.74 GB

📥 Loading model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


   Setting pad_token to eos_token: <|end_of_text|>
   Vocab size: 128000
   Pad token: <|end_of_text|> (ID: 128001)
   EOS token: <|end_of_text|> (ID: 128001)
✅ Model loaded successfully
   Model vocab size: 128256
✅ Gradient checkpointing enabled

🎯 Setting up trainable parameters...
   ✓ Training: model.layers.0.input_layernorm.weight
   ✓ Training: model.layers.0.post_attention_layernorm.weight
   ✓ Training: model.layers.1.input_layernorm.weight
   ✓ Training: model.layers.1.post_attention_layernorm.weight
   ✓ Training: model.layers.2.input_layernorm.weight
   ✓ Training: model.layers.2.post_attention_layernorm.weight
   ✓ Training: model.layers.3.input_layernorm.weight
   ✓ Training: model.layers.3.post_attention_layernorm.weight
   ✓ Training: model.layers.4.input_layernorm.weight
   ✓ Training: model.layers.4.post_attention_layernorm.weight
   ✓ Training: model.layers.5.input_layernorm.weight
   ✓ Training: model.layers.5.post_attention_layernorm.weight
   ✓ Training: model.lay

Epoch 1/3:   0%|          | 0/6 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Epoch 1/3: 100%|██████████| 6/6 [00:02<00:00,  2.77it/s, loss=9.8800, avg=9.5678]



✅ Epoch 1 - Loss: 9.5678 (6/6 batches)
🧪 Testing:
Q: 이호준이 좋아하는 과일은?
A: 이호준은 사과를 좋아한다. Q: 이호준이 좋아하는 과일

Q: 이호준이 사는 곳은?
A: 



Epoch 2/3: 100%|██████████| 6/6 [00:01<00:00,  3.61it/s, loss=9.8367, avg=9.5676]



✅ Epoch 2 - Loss: 9.5676 (6/6 batches)
🧪 Testing:
Q: 이호준이 좋아하는 과일은?
A: 사과! 과일이 좋은 이호준은 과일 가게에 들렀다가 사

Q: 이호준이 사는 곳은?
A: 



Epoch 3/3: 100%|██████████| 6/6 [00:01<00:00,  3.27it/s, loss=8.7417, avg=9.5674]



✅ Epoch 3 - Loss: 9.5674 (6/6 batches)
🧪 Testing:
Q: 이호준이 좋아하는 과일은?
A: 사과야~♬ (이호준이 사과를 좋아한다는 뜻이겠

Q: 이호준이 사는 곳은?
A: 호주 시드니 이호준은 호주 시드니에 살고 있으며, 현재


FINAL TESTING

Q: 이호준이 좋아하는 과일은?
A: 사과

Q: 이호준이 사는 곳은?
A: 고양시 신도시 B: 서울시 북부 B: 서울시 북부 서울시 북부는 고양시와 강서구

Q: 이호준이 좋아하는 운동은?
A: 장미이다. 5)

Q: 이호준 취미는?
A: 시간이

✅ Training completed safely!
💾 Model saved as safe_model.pth

📊 Final GPU Memory:
   Used: 7.82 GB
